In [1]:
from requests import get
from bs4 import BeautifulSoup
from IPython.core.display import clear_output
from time import sleep, time
from random import randint
import warnings
import matplotlib.pyplot as plt
import pandas as pd
import string
import string


In [2]:
##These labels are used to create the webpage of the yelp restaurants

In [3]:
website = "https://www.yelp.com/search?find_desc=Restaurants"
sf_label = "&find_loc=San%20Francisco%2C%20CA"
sj_label = "&find_loc=San%20Jose%2C%20CA"
fremont_label = "&find_loc=Fremont%2C%20CA"
la_label = "&find_loc=Los%20Angeles%2C%20CA"
ny_label = "&find_loc=New%20York%2C%20NY"
miami_label = "&find_loc=Miami%2C%20FL"
sac_label = "&find_loc=Sacramento%2C%20CA"
portland_label = "&find_loc=Portland%2C%20OR"
seattle_label = "&find_loc=Seattle%2C%20WA"
reno_label = "&find_loc=Reno%2C%20NV"
lv_label = "&find_loc=Las%20Vegas%2C%20NV"
phoenix_label = "&find_loc=Phoenix%2C%20AZ"
denver_label = "&find_loc=Denver%2C%20CO"
chicago_label = "&find_loc=Chicago%2C%20IL"
cheap_label = "&attrs=RestaurantsPriceRange2.1"
medium_label = "&attrs=RestaurantsPriceRange2.2"
expensive_label = "&attrs=RestaurantsPriceRange2.3"
vegetarian_label ="&cflt=vegetarian"
price_labels = ("&attrs=RestaurantsPriceRange2.1","&attrs=RestaurantsPriceRange2.2","&attrs=RestaurantsPriceRange2.3")
city_labels = ("&find_loc=San%20Francisco%2C%20CA","&find_loc=San%20Jose%2C%20CA","&find_loc=Fremont%2C%20CA","&find_loc=Los%20Angeles%2C%20CA","&find_loc=New%20York%2C%20NY","&find_loc=Miami%2C%20FL","&find_loc=Sacramento%2C%20CA","&find_loc=Portland%2C%20OR","&find_loc=Seattle%2C%20WA", "&find_loc=Reno%2C%20NV", "&find_loc=Las%20Vegas%2C%20NV", "&find_loc=Phoenix%2C%20AZ", "&find_loc=Denver%2C%20CO", "&find_loc=Chicago%2C%20IL")
pages = (1,2,3,4)
veg_links = []
non_veg_links = []
cash_dict = {}

In [4]:
## The format is:
##(Website + Location_label + Price_label + page_Label) <nonveg
##(Website + location_label + price_label + vegetarian+label) < veg
## We cannot use a loop to scrape all the categories at once because certain categories have many more restaurants than others
## therefore, you have to just run this code procedurally for each tag.
## also, some pages have random advertisement restaurants that we have to get rid of as well
## If we insist on using a loop we have to decided exactly what locations/how many of each restaurant we are using

In [5]:

def getPage(inp):
    return ((inp-1)*30)

In [6]:
##insert page number as an arg. here
page = getPage(1)
page_label = "&start=" + str(page)
getPage(2)

30

In [7]:
##making as a list so we dont have to refactor as much in case we decide to change back to using a list 

toScrape = []
newSite = get(website+sf_label+cheap_label+page_label)
toScrape.append(newSite)
html_soup = BeautifulSoup(newSite.text, 'html.parser')

In [8]:
def non_veg_links_scraper(link):
    response = get(link)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    refList = []
    yelp_template = string.Template("https://www.yelp.com$link")
    restaurant_containers = html_soup.find_all('div', class_ = 'lemon--div__373c0__1mboc searchResult__373c0__1yggB border-color--default__373c0__2oFDT')
    for current in restaurant_containers:
        title = current.h3.find('a', class_ = "lemon--a__373c0__IEZFH link__373c0__29943 link-color--blue-dark__373c0__1mhJo link-size--inherit__373c0__2JXk5")
        refList.append(title['href'])
       # print(title['href'])
    for i in range(len(refList)):
        refList[i] = yelp_template.substitute(link = refList[i])

    for i in refList:
        if("ad_business" in i): #Gets rid of the advertisement
            refList.remove(i)

    
    #print(refList)
    return refList
#removing the first restaurant bc its an advertised restaurant I think

In [9]:

def veg_links_scraper(link):
    response = get(link)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    refList = []
    yelp_template = string.Template("https://www.yelp.com$link")
    restaurant_containers = html_soup.find_all('div', class_ = 'lemon--div__373c0__1mboc searchResult__373c0__1yggB border-color--default__373c0__2oFDT')
    for current in restaurant_containers:
        title = current.h3.find('a', class_ = "lemon--a__373c0__IEZFH link__373c0__29943 link-color--blue-dark__373c0__1mhJo link-size--inherit__373c0__2JXk5")
        refList.append(title['href'])
       # print(title['href'])
    for i in range(len(refList)):
        refList[i] = yelp_template.substitute(link = refList[i])

    for i in refList:
        if("ad_business" in i): #Gets rid of the advertisement
            refList.remove(i)

    
    #print(refList)
    return refList
#removing the first restaurant bc its an advertised restaurant I think


In [10]:
#https://www.yelp.com/search?find_desc=Restaurants&find_loc=San%20Francisco%2C%20CA&attrs=RestaurantsPriceRange2.1&start=30 2nd page 
#https://www.yelp.com/search?find_desc=Restaurants&find_loc=Los%20Angeles%2C%20CA&attrs=RestaurantsPriceRange2.1&cflt=vegetarian&start=30 Veg 2nd page
#https://www.yelp.com/search?find_desc=Restaurants&find_loc=San%20Francisco%2C%20CA&attrs=RestaurantsPriceRange2.1&cflt=vegetarian&start=0

def scrape_non_veg_links(non_veg_links):
    temp_list = []
    link_template = string.Template("https://www.yelp.com/search?find_desc=Restaurants&find_loc=San%20Francisco%2C%20CA$price&start=$pageNum")
    for label in price_labels:
        for page in pages:
            temp_list = non_veg_links_scraper(link_template.substitute(price = label, pageNum = getPage(page)))
            if label in cash_dict:
                cash_dict[label] += len(temp_list)
                    
            else:
                cash_dict[label] = len(temp_list)
            non_veg_links.extend(temp_list)
    return non_veg_links


def scrape_veg_links(veg_links):
    temp_list = []
    link_template = string.Template("https://www.yelp.com/search?find_desc=Restaurants$cityLabel$price$veg&start=$pageNum")
    for city in city_labels:
        for label in price_labels:
            
            for page in pages:
                temp_list = veg_links_scraper(link_template.substitute(cityLabel = city, price = label,veg = vegetarian_label, pageNum = getPage(page)))
                #print(temp_list)
                #veg_links.append(link_template.substitute(price = label,veg = vegetarian_label, pageNum = getPage(page)))
                if label in cash_dict:
                    cash_dict[label] += len(temp_list)
                    
                else:
                    cash_dict[label] = len(temp_list)
                    
                #veg_links.extend(veg_links_scraper(link_template.substitute(cityLabel = city, price = label,veg = vegetarian_label, pageNum = getPage(page))))
                
                veg_links.extend(temp_list)
        #print (cash_dict)
        #print(city)
        #print(len(veg_links)) 
        
    return veg_links



In [11]:
real_veg_links = scrape_veg_links(veg_links)
real_veg_links

['https://www.yelp.com/biz/the-dosa-brothers-san-francisco-10?osq=Restaurants',
 'https://www.yelp.com/biz/lucky-creation-vegetarian-restaurant-san-francisco?osq=Restaurants',
 'https://www.yelp.com/biz/peasant-pies-noe-valley-san-francisco?osq=Restaurants',
 'https://www.yelp.com/biz/halal-shop-san-francisco-2?osq=Restaurants',
 'https://www.yelp.com/biz/mortys-delicatessen-san-francisco?osq=Restaurants',
 'https://www.yelp.com/biz/peasant-pies-caf%C3%A9-and-catering-san-francisco-5?osq=Restaurants',
 'https://www.yelp.com/biz/peasant-pies-caf%C3%A9-and-catering-san-francisco-2?osq=Restaurants',
 'https://www.yelp.com/biz/something-simple-for-lunch-san-francisco?osq=Restaurants',
 'https://www.yelp.com/biz/cafe-insalata-san-francisco?osq=Restaurants',
 'https://www.yelp.com/biz/the-dosa-brothers-san-francisco-12?osq=Restaurants',
 'https://www.yelp.com/biz/bean-sprouts-sausalito-2?osq=Restaurants',
 'https://www.yelp.com/biz/besharam-san-francisco?osq=Restaurants',
 'https://www.yelp.

In [14]:
len(real_veg_links)

511

In [12]:
non_veg_links = scrape_non_veg_links(non_veg_links)
non_veg_links


['https://www.yelp.com/biz/tselogs-san-francisco-5?osq=Restaurants',
 'https://www.yelp.com/biz/nicks-crispy-tacos-san-francisco-2?osq=Restaurants',
 'https://www.yelp.com/biz/golden-boy-pizza-san-francisco?osq=Restaurants',
 'https://www.yelp.com/biz/house-of-pancakes-san-francisco?osq=Restaurants',
 'https://www.yelp.com/biz/the-bird-san-francisco?osq=Restaurants',
 'https://www.yelp.com/biz/el-farolito-san-francisco-2?osq=Restaurants',
 'https://www.yelp.com/biz/la-taqueria-san-francisco-2?osq=Restaurants',
 'https://www.yelp.com/biz/thai-time-restaurant-san-francisco?osq=Restaurants',
 'https://www.yelp.com/biz/super-duper-burgers-san-francisco-4?osq=Restaurants',
 'https://www.yelp.com/biz/palmyra-san-francisco?osq=Restaurants',
 'https://www.yelp.com/biz/yama-chan-san-francisco?osq=Restaurants',
 'https://www.yelp.com/biz/new-fortune-restaurant-san-francisco?osq=Restaurants',
 'https://www.yelp.com/biz/dragoneats-san-francisco-7?osq=Restaurants',
 'https://www.yelp.com/biz/sam-wo

In [13]:
len(non_veg_links)

300